In [2]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import joblib

# Загрузка модели и энкодеров
model = xgb.XGBClassifier()
model.load_model("/Users/daniiladamenko/Documents/Курсовая Работа Адаменко ПМ22-4/KR_Adamenko_PM22-4/FinalModel.json") #путь к файлу FinalModel.json

enc = joblib.load('/Users/daniiladamenko/Documents/Курсовая Работа Адаменко ПМ22-4/KR_Adamenko_PM22-4/encoder.joblib')  # Путь к файлу encoder.joblib
le = joblib.load('/Users/daniiladamenko/Documents/Курсовая Работа Адаменко ПМ22-4/KR_Adamenko_PM22-4/label_encoder.joblib') # Путь к файлу label_encoder.joblib

selected = ['Outstanding_Debt', 'Total_EMI_per_month', 'Interest_Rate',
            'Monthly_Inhand_Salary', 'Credit_Mix', 'Delay_from_due_date',
            'Amount_invested_monthly', 'Num_Credit_Inquiries',
            'Credit_History_Age', 'Num_Credit_Card', 'Payment_of_Min_Amount',
            'Changed_Credit_Limit']

feature_order = ['Outstanding_Debt', 'Total_EMI_per_month', 'Interest_Rate', 'Monthly_Inhand_Salary', 'Credit_Mix', 'Delay_from_due_date', 'Amount_invested_monthly', 'Num_Credit_Inquiries', 'Credit_History_Age', 'Num_Credit_Card', 'Payment_of_Min_Amount', 'Changed_Credit_Limit']

enc_ui = OrdinalEncoder(categories=enc.categories, handle_unknown='use_encoded_value', unknown_value=-1)

feature_translations = {
    'Outstanding_Debt': 'Непогашенная задолженность',
    'Total_EMI_per_month': 'Общая сумма ежемесячных платежей по кредиту',
    'Interest_Rate': 'Процентная ставка',
    'Monthly_Inhand_Salary': 'Ежемесячный доход',
    'Credit_Mix': 'Качество кредитного портфеля',
    'Delay_from_due_date': 'Просрочка платежа (в днях)',
    'Amount_invested_monthly': 'Сумма ежемесячных инвестиций',
    'Num_Credit_Inquiries': 'Количество кредитных запросов',
    'Credit_History_Age': 'Возраст кредитной истории (в месяцах)',
    'Num_Credit_Card': 'Количество кредитных карт',
    'Payment_of_Min_Amount': 'Оплата минимального платежа',
    'Changed_Credit_Limit': 'Изменение кредитного лимита'
}

def predict_credit_score(button):
    input_data = {}
    for feature in selected:
        input_data[feature] = input_widgets[feature].value
    input_df = pd.DataFrame(input_data, index=[0])
    cat_features = ['Credit_Mix', 'Payment_of_Min_Amount']
    input_df[cat_features] = enc_ui.fit_transform(input_df[cat_features])
    input_df = input_df[feature_order]
    prediction = model.predict(input_df)
    dict = {0:'Хороший',1:'Плохой',2:'Стандартный'}
    credit_score = dict[prediction[0]]

    print('Предсказанный кредитный рейтинг:', credit_score) 

display(HTML("""
<style>
.widget-label {
    min-width: 300px !important;
}
</style>
"""))
input_widgets = {}

for feature in selected:
    if feature in ['Credit_Mix', 'Payment_of_Min_Amount']:
        unique_values = ['Good', 'Bad', 'Standard'] if feature == 'Credit_Mix' else ['Yes', 'No', 'NM']
        input_widgets[feature] = widgets.Dropdown(
            options=unique_values,
            description=feature_translations[feature] + ':',
            layout=widgets.Layout(width='400px')
        )
    else:
        min_val = 0
        max_val = 5000
        input_widgets[feature] = widgets.FloatText(
            description=feature_translations[feature],
            layout=widgets.Layout(width='400px')
        )

display(widgets.VBox(list(input_widgets.values())))

predict_button = widgets.Button(description="Предсказать кредитный рейтинг", layout=widgets.Layout(width='400px'))
predict_button.on_click(predict_credit_score)
display(predict_button)

/Users/daniiladamenko/anaconda3/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/daniiladamenko/anaconda3/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Button(description='Предсказать кредитный рейтинг', layout=Layout(width='400px'), style=ButtonStyle())